# Generating and Documenting a Decay Composition

This notebook provides an example workflow for:
1. Reading a TARDIS configuration and model.
2. Generating a new composition by applying nuclear decay.
3. Retrieving and examining decay radiation data such as gamma-ray lines.

It is intended as a reference on how to handle decay processes and gather the resulting radiation data in TARDIS.

In [2]:
# Step 1: Setup and Imports
import astropy.units as u
import numpy as np
import pandas as pd

from tardis.energy_input.decay_radiation import get_decay_radiation_data
from tardis.io.atom_data import AtomData
from tardis.model.matter import Composition

%matplotlib inline


/Users/wkerzend/python/tardis/tardis/__init__.py:20: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [3]:
# Step 2: Load a Configuration and Atom Data

# Provide a path to your Atom Data file:
atom_data_file = "kurucz_cd23_chianti_H_He.h5"  # Example atom data
atom_data = AtomData.from_hdf(atom_data_file)

# Create a new Composition object with 1 g/cm^3 density in a single shell of pure Ni-56
# (In a real workflow, TARDIS will build this more completely.)
nuclide_index = pd.MultiIndex.from_tuples([(28, 56)], names=["atomic_number","mass_number"])

composition = Composition(
    nuclide_mass_fraction=pd.DataFrame({0: [1.0]}, index=nuclide_index),
    density=[1.0] * (u.g / (u.cm**3)),
    element_masses=atom_data.atom_data.mass.copy(),
)

composition.density




<Quantity [1.] g / cm3>

## Step 3: Apply Decay to Generate a New Composition
Here we show how to evolve the composition by a certain time interval, reflecting radioactive decay.

In [4]:
# Example: Advance the model time by 5 days
time_epoch = 2 * u.day

# TARDIS can automatically decay the composition based on the isotopes present.
# Here we show a typical call to decay methods:
decayed_mass_fractions = composition.isotopic_mass_fraction.calculate_decayed_mass_fractions(time_epoch)

# Show the updated isotopic mass fractions:
decayed_mass_fractions.head()

,,0
atomic_number,mass_number,
26,56,0.001889
27,56,0.202135
28,56,0.795968


## Step 4: Retrieve Decay Radiation Data
The `atom_data` object includes lines and energies emitted during nuclear decay. We examine them here.

In [5]:
# Retrieve decay radiation data:
em_radiation_data, bp_radiation_data = get_decay_radiation_data(
    atom_data.decay_radiation_data, composition.isotopic_mass_fraction.index
)

display(em_radiation_data.head())
display(bp_radiation_data.head())

## Confer Dutta et al. 2025 Table 1 for the decay data used here
em_radiation_data.loc[28, 56,12]


radiation_energy_kev  \
atomic_number mass_number channel_id                         
27            56          27                       511.000   
                          28                         6.404   
                          29                         6.391   
                          30                         7.058   
                          31                         7.058   

                                      energy_per_decay_kev radiation_type  
atomic_number mass_number channel_id                                       
27            56          27                    199.290000       Annihil.  
                          28                      0.928580         XR ka1  
                          29                      0.466543         XR ka2  
                          30                      0.121398         XR kb1  
                          31                      0.062110         XR kb3

radiation_energy_kev  \
atomic_number mass_number channel_id                         
27            56          0                           45.3   
                          1                           76.7   
                          2                          178.7   
                          3                          247.1   
                          4                          631.2   

                                      energy_per_decay_kev radiation_type  
atomic_number mass_number channel_id                                       
27            56          0                       0.003715            NaN  
                          1                       0.000046            NaN  
                          2                       1.860267            NaN  
                          3                       0.016556            NaN  
                          4                     116.140800            NaN

radiation_energy_kev       158.38
energy_per_decay_kev    156.47944
radiation_type                NaN
Name: (28, 56, 12), dtype: object

## Step 5: Calculate Number of Decays
We can also calculate the number of decays over a given time period for each shell.
This can be useful for understanding the decay process in more detail.

In [6]:
# using volume to be consistent with Dutta et al. 2025
volume = (1.3545205e30) * u.cm**3
cell_masses = composition.calculate_cell_masses(volume)
# Example: Calculate the number of decays over  0.09 days (cf. Dutta et al. 2025 Table 1)
number_of_decays = decayed_mass_fractions.calculate_number_of_decays(0.094257 * u.day, cell_masses)

# Show the number of decays:
number_of_decays.head()


,cell_id,0
atomic_number,mass_number,
27,56,2.545038e+48
28,56,1.241517e+50


## Step 6: Summarize Decay Energies

In this cell, we multiply the energy emitted per decay (in keV)
by the number of decays in each shell to determine the total
energy (in keV) contributed by nuclear decay per channel in each shell.
Just like the previous steps, we display the result as a DataFrame
indexed by isotopes and with columns representing the shells.

In [7]:
decay_energy = pd.DataFrame(
    em_radiation_data["energy_per_decay_kev"].values[:, None]
    * number_of_decays.reindex(em_radiation_data.index.droplevel(2)).values,
    index=em_radiation_data.index,
    columns=number_of_decays.columns
)

#display(decay_energy.head())
## Confer Dutta et al. 2025 Table 1 for the decay data used here
decay_energy.loc[28, 56, 12]


cell_id
0    1.942718e+52
Name: (28, 56, 12), dtype: float64

In [8]:

time_start = 2 * u.day
decayed_mass_fractions = composition.isotopic_mass_fraction.calculate_decayed_mass_fractions(time_start)
all_decay_energy = []
for time in np.linspace(time_start, 200 * u.day, 200):
    time_delta = 1 * u.day
    decayed_mass_fractions = decayed_mass_fractions.calculate_decayed_mass_fractions(time_delta)
    number_of_decays = decayed_mass_fractions.calculate_number_of_decays(time_delta, cell_masses)
    all_decay_energy.append(pd.DataFrame(
        em_radiation_data["energy_per_decay_kev"].values[:, None]
        * number_of_decays.reindex(em_radiation_data.index.droplevel(2)).values,
        index=em_radiation_data.index))

x = pd.concat(all_decay_energy)

In [11]:
x.sample(weights=x[0], n=10000)

0
atomic_number mass_number channel_id              
28            56          15          4.144926e+53
27            56          51          8.667681e+52
                          62          1.372503e+52
                          51          8.489408e+52
                          40          4.580369e+52
...                                            ...
                          78          1.940563e+49
                          41          1.228449e+49
                          35          9.842633e+48
                          35          8.917336e+48
                          37          8.316877e+48

[10000 rows x 1 columns]

## Conclusion
This notebook has demonstrated how to:
1. Load a TARDIS model and atom data.
2. Apply radioactive decay to the composition.
3. Retrieve and inspect the decay-related energies and lines.

Use this script as a foundation for your own deeper investigations into nebular or high-energy phenomena in TARDIS!